## Importing packages

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from scipy import stats as st

cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 12

## Definition of variables

In [ ]:
lifet = # Add your code #
m = # Add your code #
n = # Add your code #

# number of samples
samples = # Add your code #
#fixing the seed for reproducibility
np.random.seed(10)

## Initial crack size ##
a0_mean = # Add your code #

## Geometric factor ##
Y_mean = # Add your code #
Y_std = # Add your code #
mu_Y = np.log((Y_mean**2)/math.sqrt(Y_std**2+Y_mean**2))
sigma_Y = math.sqrt(np.log(Y_std**2/(Y_mean**2)+1))

## Paris' law parameter: crack growth ##
logCmean = # Add your code #;
logCstd = # Add your code #;

## Stress range ##
Se_mean = # Add your code #
Se_cov = # Add your code #
Se_std = Se_cov*Se_mean
mu_Se = np.log((Se_mean**2)/math.sqrt(Se_std**2+Se_mean**2))
sigma_Se = math.sqrt(np.log(Se_std**2/(Se_mean**2)+1))

## Thickness ##
a_c = # Add your code #

### 'Pseudo' random number generation

In [ ]:
## Scale == mean
a0_rn = np.random.exponential(scale=a0_mean, size=(samples))
print('a0   | Mean: ', np.mean(a0_rn), '  Std: ', np.std(a0_rn))

Y_rn  = np.random.lognormal(mu_Y, sigma_Y, size=(samples))
print('Y    | Mean: ', np.mean(Y_rn), '  Std: ', np.std(Y_rn))

logC_rn  = np.random.normal(logCmean, logCstd, size=(samples))
print('logC | Mean: ', np.mean(logC_rn), '  Std: ', np.std(logC_rn))
C_rn = 10**logC_rn
print('C    | Mean: ', np.mean(C_rn), '  Std: ', np.std(C_rn))

Se_rn  = np.random.lognormal(mu_Se, sigma_Se, size=(samples))
print('Se   | Mean: ', np.mean(Se_rn), '  Std: ', np.std(Se_rn))

## Stochastic crack size computation
Crack growth model => Paris' law

$a_{t+1} = [a_{t}^{(2-m)/2} + (2-m)/2)C(Y_\pi^{0.5}Se)^mn]^{2/(2-m)}$

In [ ]:
a0_it = a0_rn.copy()
a_it = np.zeros((samples, lifet))
for i in range(lifet):
    a_it[:, i] = (a0_it**((2-m)/2) + ((2-m)/2)*C_rn*(Y_rn*math.pi**0.5*Se_rn)**m*n)**(2/(2-m))
    ## For numerical stability purposes ##
    a_it[a_it > a_c] = a_c + 1 
    C_rn[a_it[:,i] > a_c] = 1e-100
    ##--------------------------------- ##
    
    ## Verbosity ##
#     print('a_it | Mean: ', np.mean(a_it[:, i]), '  Std: ', np.std(a_it[:, i]))
#     print('p_f: ', p_f[i])
    ##-----------##
    a0_it = a_it[:, i]

In [ ]:
time_ = np.arange(1, lifet+1)
fig, ax = plt.subplots(1, figsize=(14*cm, 8*cm))
ax.set_title('Crack growth', pad=1)
ax.set_xlabel('Time (years)')
ax.set_ylabel('$a (mm)$')
ax.set_ylim([0,20])
for j in range(200):
    ax.plot(time_, a_it[j,:], color='grey')
plt.subplots_adjust(left=0.10, right=.98, top=0.90, bottom=0.20, hspace = 0.65, wspace=0.15)
plt.show()

## Failure probability computation

In [ ]:
g_fail = # Add your code #
failures = g_fail <= 0
failures_n = np.sum(failures, axis=0)
pf = failures_n / samples
print('Failure probability | Year :', lifet, '=> ', pf[lifet-1])

## Verbosity ##
# print('Failures: ', failures_n)
# print('Failure probability: ', pf)
##-----------##

time_ = np.arange(1, lifet+1)
fig, ax = plt.subplots(1, figsize=(17*cm, 8*cm))
ax.set_yscale('log') # Log-scale #
ax.set_title('Failure probability computation', pad=1)
ax.set_xlabel('Time (years)')
ax.set_ylabel('$P_F$')
ax.plot(time_, pf)
plt.subplots_adjust(left=0.10, right=.98, top=0.90, bottom=0.20, hspace = 0.65, wspace=0.15)
plt.show()